In [1]:
import numpy as np

# Kyber simplified parameters for demonstration
KYBER_N = 256  # Polynomial degree
KYBER_Q = 3329 # Prime modulus
KYBER_SHARED_KEY_SIZE = 32  # Length of the shared key in bytes

# Helper functions for polynomial operations
def generate_polynomial():
    """Generates a random polynomial with coefficients in range [0, KYBER_Q)."""
    return np.random.randint(0, KYBER_Q, KYBER_N)

def centered_binomial_sample(size):
    """Generates centered binomial noise, commonly used in lattice-based cryptography."""
    return np.sum(np.random.randint(0, 2, (size, 2)), axis=1) - 1

def poly_add(a, b):
    """Adds two polynomials modulo KYBER_Q."""
    return (a + b) % KYBER_Q

def poly_sub(a, b):
    """Subtracts two polynomials modulo KYBER_Q."""
    return (a - b) % KYBER_Q

def poly_mult(a, b):
    """Multiplies two polynomials in R_q, reducing modulo KYBER_Q (simplified)."""
    result = np.convolve(a, b) % KYBER_Q
    return result[:KYBER_N]

# Key generation
def generate_keypair():
    """Generates a Kyber KEM public/private key pair."""
    sk = generate_polynomial()                  # Private key (secret polynomial)
    pk = poly_mult(generate_polynomial(), sk)   # Public key derived from sk
    e = centered_binomial_sample(KYBER_N)       # Noise for public key
    pk = poly_add(pk, e)                        # Public key: pk = a * sk + e
    return pk, sk

# Encapsulation - generates a ciphertext and a shared key
def encapsulate(pk):
    """Encapsulates a shared key using the public key, producing ciphertext and shared key."""
    shared_key = np.random.bytes(KYBER_SHARED_KEY_SIZE)  # Generate shared key as random bytes

    # Convert shared key to a polynomial for encryption
    message_poly = np.frombuffer(shared_key, dtype=np.uint8)
    message_poly = np.pad(message_poly, (0, KYBER_N - len(message_poly)), 'constant')
    
    # Encryption process
    r = centered_binomial_sample(KYBER_N)           # Random polynomial for encapsulation
    e1 = centered_binomial_sample(KYBER_N)
    e2 = centered_binomial_sample(KYBER_N)
    
    u = poly_add(poly_mult(r, pk), e1)              # u = r * pk + e1
    v = poly_add(poly_mult(r, message_poly), e2)    # v = r * message + e2
    
    ciphertext = (u, v)                             # Ciphertext (u, v)
    return ciphertext, shared_key

# Decapsulation - decrypts the ciphertext using the private key to obtain the shared key
def decapsulate(ciphertext, sk):
    """Decapsulates the shared key using the private key."""
    u, v = ciphertext
    recovered_message_poly = poly_sub(v, poly_mult(u, sk))  # m = v - u * sk
    
    # Convert recovered polynomial back to bytes
    recovered_shared_key = recovered_message_poly[:KYBER_SHARED_KEY_SIZE].astype(np.uint8).tobytes()
    return recovered_shared_key

# Example usage
if __name__ == "__main__":
    # Key generation
    pk, sk = generate_keypair()
    print("Public Key:", pk)
    print("Private Key:", sk)

    # Encapsulate to create a ciphertext and shared key
    ciphertext, shared_key = encapsulate(pk)
    print("\nOriginal Shared Key:", shared_key)
    print("Ciphertext (u, v):", ciphertext)

    # Decapsulate to retrieve the shared key
    recovered_shared_key = decapsulate(ciphertext, sk)
    print("\nRecovered Shared Key:", recovered_shared_key)

    # Check if decapsulation is correct
    print("\nDecapsulation Successful:", shared_key == recovered_shared_key)


Public Key: [3076 2870 1260 2875  797  565 2439 2499 1124 1671  857 3167  248 2227
 1173 3105  328 1358 3074  861 1055 2646 1994 1292 2401  758 2396  159
 2459   79  384  659 1704 1506 1567 2680 1928  156 1326 1274 1966 3204
 2639 2833   11  648 2395  634 1353  147 2394 1496 1495 1179   18 1172
 1940 1789 2789  555 1465 1166  713  161 1327 1037 2994 2966 1676  480
  420 2882 3245 2017 2224  631   82 3270  411 2721 3063  478  865 2425
 1238 1875 2210 2596 2795 1299 2873 2589 1956 2644 2988 3225 2308 3236
 3108 1992  778 1661 3060 3164 1047 1585 2794 3111 1279  888  786 2859
 1142 1219 1222 3037 1910 2445 2938 1971 1438 3286 1653 2220  994 2874
 1741 1148  638 1848  417  299 3163   95 2800  680 3104  628  313 1373
  519 2452  966 1113 1803  630 1170  689  398 2075  656 2702 2890 1957
 1077  335 1116  563  385 2483 1355  442   82 1736 1844 2047  184 2398
 3074  862 2058 1063 1752  344 2534 1620 2549 1224  455 2433 1572 1941
 1541  399 2249 1235 2660 1526 1037 1323 1130  319 2238  517  464

In [2]:
pip install numpy

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 375.8 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 361.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.9 MB 2.0 MB/s eta 0:00:07
   ---------------------------------------- 0.1/12.9 MB 1.1 MB/s eta 0:00:12
    --------------------------------------- 0.2/12.9 MB 1.8 MB/s eta 0:00:08
   - -------------------------------------- 0.4/12.9 MB 2.1 MB/s eta 0:00:06
   - -------------------------------------- 0.5/12.9 MB 2.4 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/12.9 MB 3.1 MB/s eta 0:00:04
   --- ------------------------------------ 1.1/12.9 MB 3.2 MB/s eta 0:00:04


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np

# Define parameters
KYBER_K = 2  # This would change based on the security level
MODULUS_Q = 3329
POLY_SIZE = 256  # Size of polynomials in Kyber

# Step 1: Polynomial arithmetic functions
def poly_add(a, b):
    """Addition of polynomials in modular field."""
    return (a + b) % MODULUS_Q

def poly_mult(a, b):
    """Multiplication of polynomials in modular field."""
    # This would be a polynomial multiplication with modular reduction
    # For simplicity, let's assume it's a basic numpy multiplication mod q
    result = np.convolve(a, b) % MODULUS_Q
    return result[:POLY_SIZE]

# Step 2: Key generation
def keygen():
    """Generate public and private keys."""
    # Sample polynomials for private key
    sk = np.random.randint(0, MODULUS_Q, size=(KYBER_K, POLY_SIZE))
    # Create a public key using sk and other Kyber processes
    pk = np.random.randint(0, MODULUS_Q, size=(KYBER_K, POLY_SIZE))  # Placeholder
    return pk, sk

# Step 3: Encryption
def encrypt(pk, message):
    """Encrypt a message with the public key."""
    # Encode message into polynomial format
    m_poly = np.array(message) % MODULUS_Q
    # Encryption steps with public key and noise polynomials
    c1 = poly_mult(pk[0], m_poly)  # Simplified
    c2 = poly_mult(pk[1], m_poly)  # Simplified
    return (c1, c2)

# Step 4: Decryption
def decrypt(sk, ciphertext):
    """Decrypt a ciphertext with the private key."""
    c1, c2 = ciphertext
    # Decryption operation using private key sk
    m_poly = poly_mult(sk[0], c1) - poly_mult(sk[1], c2)  # Simplified
    return m_poly % MODULUS_Q

# Test the implementation
pk, sk = keygen()
message = [1, 0, 1, 1] + [0] * (POLY_SIZE - 4)  # Simple binary message in polynomial form
ciphertext = encrypt(pk, message)
decrypted_message = decrypt(sk, ciphertext)

print("Original Message:", message[:10])
print("Decrypted Message:", decrypted_message[:10])


Original Message: [1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
Decrypted Message: [ 528  988 2252 2788 2202 2381    6  470 2135 3286]


In [3]:
import numpy as np

# Parameters for simplicity
KYBER_K = 2
MODULUS_Q = 3329
POLY_SIZE = 256

# Step 1: Polynomial Arithmetic
def poly_add(a, b):
    """Addition of polynomials in modular field."""
    return (a + b) % MODULUS_Q

def poly_mult(a, b):
    """Multiplication of polynomials in modular field."""
    result = np.convolve(a, b) % MODULUS_Q
    return result[:POLY_SIZE]

# Step 2: Key Generation
def keygen():
    """Generate public and private keys."""
    sk = np.random.randint(0, MODULUS_Q, size=(KYBER_K, POLY_SIZE))
    pk = np.random.randint(0, MODULUS_Q, size=(KYBER_K, POLY_SIZE))
    return pk, sk

# Step 3: Encapsulation
def encapsulate(pk):
    """Encapsulate to produce a ciphertext and a shared secret."""
    # Generate a random message polynomial for shared key
    message = np.random.randint(0, MODULUS_Q, size=POLY_SIZE)
    
    # Perform encryption operations (simplified for illustration)
    c1 = poly_mult(pk[0], message)
    c2 = poly_mult(pk[1], message)
    ciphertext = (c1, c2)
    
    # Derive a shared key from the message (simplified hash-like operation)
    shared_key = np.sum(message) % MODULUS_Q  # Simplified
    return ciphertext, shared_key

# Step 4: Decapsulation
def decapsulate(sk, ciphertext):
    """Decapsulate to retrieve the shared key."""
    c1, c2 = ciphertext
    message_poly = poly_mult(sk[0], c1) - poly_mult(sk[1], c2)
    message_poly = message_poly % MODULUS_Q
    
    # Derive shared key from the decrypted message
    shared_key = np.sum(message_poly) % MODULUS_Q  # Simplified
    return shared_key

# Testing the flow
pk, sk = keygen()
ciphertext, encapsulated_shared_key = encapsulate(pk)
decapsulated_shared_key = decapsulate(sk, ciphertext)

print("Ciphertext:", ciphertext)
print("Encapsulated Shared Key:", encapsulated_shared_key)
print("Decapsulated Shared Key:", decapsulated_shared_key)

# Validation
print("Keys match:", encapsulated_shared_key == decapsulated_shared_key)


Ciphertext: (array([ 173, 2813, 3051, 1890, 2981, 3077, 2501,  182, 2355, 2874, 2759,
       2280, 1976,    9, 2175, 2753, 1537,  934,  537, 3179, 2693,  828,
       1193, 1779,  140, 2165, 1391,  868, 2294,  518, 3158, 1492, 2427,
       1861,  317,  580, 1694, 1595, 1887, 1343, 2516, 1505, 2308, 2784,
       2819,  312,  511, 1807,  424, 3209, 1766, 2061,  432, 1923, 2280,
       2891,  267, 2606, 3154, 2141, 2596, 2361,  542, 1241, 1421,  330,
       3092, 1111, 3307, 2280, 2709,  641, 1243, 2214, 1245, 1752, 2649,
       2419,  394, 2136, 3192, 3134, 3042, 1029, 2339,   40, 1158, 2556,
       1037, 1094,  934,  465, 1902, 2537,  184, 3057,  960, 1188, 2384,
       1397, 2491, 3282,  109, 2167, 1909, 3311, 2497, 1777, 1679, 1326,
        977, 1461,  716, 1028, 3123,  288, 1578,  888,  285, 2640, 1592,
        453, 1135, 2112,  168, 2993, 2737, 1117, 2264,  313,  822, 2801,
       1732,  708, 2152, 3290, 2896, 1612, 2001, 2049,  292,  323, 2418,
       2446, 1656, 2590,  741, 1705, 3

In [4]:
import numpy as np

# Parameters for modular arithmetic in this example (simplified)
KYBER_K = 2
MODULUS_Q = 3329
POLY_SIZE = 256

# Polynomial Arithmetic
def poly_add(a, b):
    """Addition of polynomials with modular reduction."""
    return (a + b) % MODULUS_Q

def poly_mult(a, b):
    """Polynomial multiplication with modular reduction."""
    result = np.convolve(a, b) % MODULUS_Q
    return result[:POLY_SIZE]

# Key Generation
def keygen():
    """Generate public and private keys."""
    sk = np.random.randint(0, MODULUS_Q, size=(KYBER_K, POLY_SIZE))
    pk = np.random.randint(0, MODULUS_Q, size=(KYBER_K, POLY_SIZE))
    return pk, sk

# Encapsulation
def encapsulate(pk):
    """Encapsulate to produce a ciphertext and a shared key."""
    # Generate a random message polynomial for the shared key
    message = np.random.randint(0, MODULUS_Q, size=POLY_SIZE)
    
    # Encrypt message using the public key
    c1 = poly_mult(pk[0], message)
    c2 = poly_mult(pk[1], message)
    ciphertext = (c1, c2)
    
    # Derive the shared key (using sum of message as a placeholder)
    shared_key = int(np.sum(message) % MODULUS_Q)  # Simplified shared key
    return ciphertext, shared_key

# Decapsulation
def decapsulate(sk, ciphertext):
    """Decapsulate to retrieve the shared key."""
    c1, c2 = ciphertext
    decrypted_message = poly_mult(sk[0], c1) - poly_mult(sk[1], c2)
    decrypted_message = decrypted_message % MODULUS_Q
    
    # Derive the shared key from the decrypted message
    shared_key = int(np.sum(decrypted_message) % MODULUS_Q)
    return shared_key

# Testing the flow
pk, sk = keygen()
ciphertext, encapsulated_shared_key = encapsulate(pk)
decapsulated_shared_key = decapsulate(sk, ciphertext)

print("Ciphertext:", ciphertext)
print("Encapsulated Shared Key:", encapsulated_shared_key)
print("Decapsulated Shared Key:", decapsulated_shared_key)

# Check if the shared keys match
print("Keys match:", encapsulated_shared_key == decapsulated_shared_key)


Ciphertext: (array([ 311, 1849, 2448, 2045, 1108, 2977,  675,  903, 1027,  166, 2868,
       1203, 3298, 2789, 2858, 2569, 2292, 2277,  722, 2878, 3296, 3132,
       2389, 1631, 2548, 2431,  763, 2583, 1568,  940, 1601, 2049, 1959,
       2512, 1247, 1416, 3081,  646, 1975, 1296, 3181, 1505, 3032,  604,
        603, 1465, 1257,  368,  526, 1758,  660, 1009, 1162, 1323, 1938,
       3006, 1468,  525,   18,  393, 1875,  427, 2085, 2234,  539, 1741,
       1572,  677,  689, 1481, 2430, 1081, 2284, 1157, 1988,  229, 1013,
       2056,  138, 2670, 2611,  785,  882, 2541, 1576, 2734,  881,  322,
        342, 2081, 1729, 1088, 3108, 3199,  676,  144, 2034, 2166, 1048,
       1557, 1558, 1012,  357, 2459, 3254,  233, 1851,  246, 1723, 2921,
        206,  797, 2149, 1124, 2882,  848, 1375, 2235, 1250,  588,  503,
        111, 2914, 2206, 1147,  392, 2516, 2936, 1907,  871, 3179,  468,
        694, 2199,  875,  452, 3151, 1523, 2422,   48, 2818, 3140,  692,
       1919, 2159,  248, 1080, 2601, 1

In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
